###Unidade 4  - Recuperação da Informação
Aluno(a): Tainá da Silva Lima
<br>
DRE: 116165607
<br><br>
Aluno(a): Rafael Pais Cardoso
<br>
DRE: 116140788
<br><br>
Professor(a): Giseli Lopes
<br>
Período: 2020.4 PLE

# Importando as bibliotecas necessárias aos exercícios realizados nesse trabalho

In [ ]:
import numpy as np

# Exercício 1

Enunciado:
*   Implemente as etapas de pré-processamento de documentos e consulta a seguir:
  1.   Tokenizar os documentos utilizando os separadores
especificados na entrada
  2.   Normalizar termos (ex. caixa-baixa)
  3.   Eliminar stopwords especificadas na entrada
*   Criar uma “variante” de matriz de incidências (obs.:
ao invés de apenas 0s e 1s, guarde, na matriz, a
frequência de aparecimento dos termos em cada
documento)


In [ ]:
# M             --> Vetor coluna onde cada linha representa o texto de um documento (matriz Nx1).
# separadores   --> Vetor linha de caracteres (matriz 1xNc), onde o elemento em cada coluna representa um separador a ser usado na tokenização dos documentos.
# stopwords     --> Vetor linha de strings (matriz 1xNs), onde o elemento em cada coluna armazena uma stopword.
# Essa função aplica as etapas de pré-processamento nos documentos.
def normalizacaoTokenizacao(M, separadores, stopwords):

  matriz = []

  for index in range(0, len(M)):

    # Recebe o texto de um novo documento a cada iteração.
    documentoTemporario = M[index][0]

    # Remoção dos delimitadores e normalização.
    for delimitador in separadores:
      documentoTemporario = documentoTemporario.lower().replace(delimitador, " ")

    # Obtenção dos possíveis tokens
    tokens = documentoTemporario.split()

    # Remoção das stopwords e armazenamento dos tokens em uma matriz Nx1 (cada linha representa um documento e a coluna apresenta a lista de tokens extraídos do documento).
    matriz.append(list(filter(lambda a: a not in stopwords, tokens)))
    
  return matriz

In [ ]:
# matriz --> matriz de tokens Nx1 (cada linha representa um documento e a coluna apresenta a lista de tokens extraídos do documento).
# Essa função cria a matriz de incidências. Essa matriz indica se um determinado token existe (valor 1) ou não (valor 0) em um documento, assim como armazena a frequência total
# com que os tokens aparecem na coleção.
def criarMatrizIncidencias(matriz):

  # A chave do diconário é o token. O valor associado à chave é uma matriz 1x2, em que o elemento dicionarioTokens[token][0][0] representa uma matriz 1xN, sendo que cada coluna está 
  # associada a um respectivo documento e os possíveis valores que podem assumir são 0 e 1 (o valor 0 significada que o token não se encontra no documento e o valor 1 significa que o
  # token se encontra no documento). Assim, se temos o valor 0 na segunda coluna dicionarioTokens[token][0][1]) e o elemento matriz[0][1] é a frequência total.
  dicionarioTokens = {}
  
  # Itera sobre os documentos
  for index in range(0, len(matriz)):

    # Itera em cada elemento da lista de token
    for token in matriz[index]:

      # Adiciona o token ao dicionário, caso ele não exista. Além disso, seta o valor para 1 para o documento ao qual pertence e atualiza o valor da frequência.
      if(token not in dicionarioTokens.keys()):
        x                       = [0]*len(matriz)
        x[index]                = 1
        dicionarioTokens[token] = [x,1]
        
      # Se o token já existe e o documento ao qual pertence está com o valor 0, então seta o valor para 1 e incrementa-se a frequência total.
      else:
        if(dicionarioTokens[token][0][index] == 0):
          dicionarioTokens[token][0][index] = 1
          dicionarioTokens[token][1]        +=1
  return dicionarioTokens



In [ ]:
# matriz --> matriz de tokens Nx1 (cada linha representa um documento e a coluna apresenta a lista de tokens extraídos do documento).
# Essa função cria a matriz de incidências. Essa matriz indica se um determinado token existe em um documento (valor > 0 caso exista. Do contrário, valor = 0), assim como
# a frequência neste documento.
def criarMatrizIncidenciasV2(matriz):

  # A chave do diconário é o token. O valor associado à chave é uma matriz 1x1, em que o elemento dicionarioTokens[token][0][0] representa uma matriz 1xN, sendo que cada coluna está 
  # associada a um respectivo documento e os possíveis valores que podem assumir são maiores ou iguais a 0 (o valor 0 significada que o token não se encontra no documento e o valores
  # maiores do que 0 significam a frequência com aparecem no documento.
  dicionarioTokensV2 = {}

  # Itera sobre os documentos
  for index in range(0, len(matriz)):
    
    # Itera em cada elemento da lista de token
    for token in matriz[index]:

      # Adiciona o token ao dicionário, caso ele não exista. Além disso, seta o valor para 1 para o documento ao qual pertence.
      if(token not in dicionarioTokensV2.keys()):
        x                         = [0]*len(matriz)
        x[index]                  = 1
        dicionarioTokensV2[token] = [x]

      # Incrementa-se a frequência com que aparece no respectivo documento
      else:
          dicionarioTokensV2[token][0][index] += 1
          
  return dicionarioTokensV2


# Exercício 2

Enunciado

*   Sua implementação deve:
  1.   Partindo da solução de implementação anterior (pré-processamento)
  2.   Usar uma solução de indexação utilizando uma variação da matriz de incidências (obs.: guarde a frequência de aparecimento dos termos em cada documento)
  3.   Responder consultar puramente conjuntivas e disjuntivas 
    * AND entre todos os termos da consulta
    * OR entre todos os termos da consulta



In [ ]:
# Essa função aplica as etapas de pré-processamento na consulta.
def tratamentoConsulta(consulta, separadores, stopwords):

  consultaTratada = consulta
  for delimitador in separadores:
      consultaTratada = consultaTratada.lower().replace(delimitador, " ")
  consultaTratada = consultaTratada.split()

  consultaTratada = list(filter(lambda a: a not in stopwords, consultaTratada))
  
  return consultaTratada

# Recupera-se os documentos associados aos tokens extraídos da consulta. Caso um token não exista na nossa coleção, o retiramos da consulta.
def recuperaDocumento(consulta, operacao, matrizIncidencia, colecao, separadores, stopwords):
    
    consultaTratada = tratamentoConsulta(consulta, separadores, stopwords)
    termosAusentes = ""
    for termo in consultaTratada:
      if (termo in consultaTratada) and (termo not in matrizIncidencia.keys()):
        consultaTratada.remove(termo)
        termosAusentes += termo + ", "
    
    if (len(termosAusentes) > 0):
      termosAusentes = termosAusentes[:(len(termosAusentes)-2)] 
      print("Os termos: " + termosAusentes + " não estão mapeados em nossa coleção, contudo iremos retornar os documentos cujos os outros termos estão presentes.")

    resultado = []
    if (len(consultaTratada) > 0) and (consultaTratada[0] in matrizIncidencia.keys()):
      resultado = matrizIncidencia[consultaTratada[0]][0]
      
      if (len(consultaTratada) > 1):
        for i in range(1,len(consultaTratada)):
          if (i <= len(consultaTratada)-1):
            if (consultaTratada[i] in matrizIncidencia.keys()):
              listaDeIncidencias = matrizIncidencia[consultaTratada[i]][0]
              if (operacao == 1):
                resultado =  [ resultado[i] & (1 if listaDeIncidencias[i] !=0 else 0) for i in range(len(listaDeIncidencias)) ] 
              elif (operacao == 2):
                resultado =  [ resultado[i] | (1 if listaDeIncidencias[i] !=0 else 0) for i in range(len(listaDeIncidencias)) ]  
              
    else:
      print("A consulta possui somente um termo e este não está mapeado em nossa coleção.")

    documentosResultado = []
    for j in range(len(resultado)):
      if (resultado[j] != 0):
        documentosResultado.append(colecao[j])
    
    return documentosResultado
          


# Exemplos

## Exemplo 1

In [ ]:
import numpy as np
# Conjunto de documentos
M1            = [["O peã e o caval são pec de xadrez. O caval é o melhor do jog."],
                ["A jog envolv a torr, o peã e o rei."],
                ["O peã lac o boi"],
                ["Caval de rodei!"],
                ["Polic o jog no xadrez."]]

# Lista de stopwords
stopwords1    = ["a", "o", "e", "é", "de", "do", "no", "são"]

# String contendo os termos da consulta
q1            = "xadrez peã caval torr"

# Separadores para a tokenizacao
separadores1  = [" ",",",".","!","?"] 

In [ ]:
resultado = normalizacaoTokenizacao(M1, separadores1, stopwords1)
print("Resultado da normalização e tokenização:")
print(resultado)
print("\n")

resultado2 = criarMatrizIncidencias(resultado)
print("Resultado da criação da matriz de incidências (versão em que frequência = nº de vezes que aparece na coleção):")
print(resultado2)
print("\n")

resultado22 = criarMatrizIncidenciasV2(resultado)
print("Resultado da criação da matriz de incidências 2 (versão em que frequência = nº de vezes que aparece em cada documento):")
print(resultado22)
print("\n")

print("Resultado da recuperação dos documentos dado uma consulta, operação, matriz de incidências e coleção:")
docsRecuperados = recuperaDocumento(q1, 1, resultado22, M1, separadores1, stopwords1)
print(docsRecuperados)

Resultado da normalização e tokenização:
[['peã', 'caval', 'pec', 'xadrez', 'caval', 'melhor', 'jog'], ['jog', 'envolv', 'torr', 'peã', 'rei'], ['peã', 'lac', 'boi'], ['caval', 'rodei'], ['polic', 'jog', 'xadrez']]


Resultado da criação da matriz de incidências (versão em que frequência = nº de vezes que aparece na coleção):
{'peã': [[1, 1, 1, 0, 0], 3], 'caval': [[1, 0, 0, 1, 0], 2], 'pec': [[1, 0, 0, 0, 0], 1], 'xadrez': [[1, 0, 0, 0, 1], 2], 'melhor': [[1, 0, 0, 0, 0], 1], 'jog': [[1, 1, 0, 0, 1], 3], 'envolv': [[0, 1, 0, 0, 0], 1], 'torr': [[0, 1, 0, 0, 0], 1], 'rei': [[0, 1, 0, 0, 0], 1], 'lac': [[0, 0, 1, 0, 0], 1], 'boi': [[0, 0, 1, 0, 0], 1], 'rodei': [[0, 0, 0, 1, 0], 1], 'polic': [[0, 0, 0, 0, 1], 1]}


Resultado da criação da matriz de incidências 2 (versão em que frequência = nº de vezes que aparece em cada documento):
{'peã': [[1, 1, 1, 0, 0]], 'caval': [[2, 0, 0, 1, 0]], 'pec': [[1, 0, 0, 0, 0]], 'xadrez': [[1, 0, 0, 0, 1]], 'melhor': [[1, 0, 0, 0, 0]], 'jog': [[1, 1, 0,

## Exemplo 2

In [ ]:
# Conjunto de documentos
M2              = [["Parasita é o grande vencedor do Oscar 2020, com quatro prêmios"],
                  ["Green Book, Roma e Bohemian Rhapsody são os principais vencedores do Oscar 2019"],
                  ["Oscar 2020: Confira lista completa de vencedores. Parasita e 1917 foram os grandes vencedores da noite"],
                  ["Em boa fase, Oscar sonha em jogar a Copa do Mundo da Rússia"],
                  ["Conheça os indicados ao Oscar 2020; Cerimônia de premiação acontece em fevereiro"],
                  ["Oscar Schmidt receberá Troféu no Prêmio Brasil Olímpico 2019. Jogador de basquete com mais pontos em Jogos Olímpicos."],
                  ["Seleção brasileira vai observar de 35 a 40 jogadores para definir lista da Copa América"],
                  ["Oscar 2020: saiba como é a escolha dos jurados e como eles votam"],
                  ["Bem, Amigos! discute lista da Seleção, e Galvão dá recado a Tite: Cadê o Luan?"],
                  ["IFAL-Maceió convoca aprovados em lista de espera do SISU para chamada oral"],
                  ["Arrascaeta e Matías Viña são convocados pelo Uruguai para eliminatórias da Copa. Além deles, há outros destaques na lista."],
                  ["Oscar do Vinho: confira os rótulos de destaque da safra 2018"],
                  ["Parasita é o vencedor da Palma de Ouro no Festival de Cannes"],
                  ["Estatísticas. Brasileirão Série A: Os artilheiros e garçons da temporada 2020"],
                  ["Setembro chegou! Confira o calendário da temporada 2020/2021 do futebol europeu"]]

# Lista de stopwords
stopwords2     = ["a", "o", "e", "é", "de", "do", "da", "no", "na", "são", "dos", "com", "como", "eles", "em", "os", "ao", "para", "pelo"] 

# String contendo os termos da consulta
q2             = "oscar \de 2020"

# Separadores para a tokenizacao
separadores2   = [" ",",",".","!","?",":","/","\\",";" ] 

In [ ]:
resultadoEx2 = normalizacaoTokenizacao(M2, separadores2, stopwords2)
print("Resultado da normalização e tokenização:")
print(resultadoEx2)
print('\n')

resultado2Ex2 = criarMatrizIncidencias(resultadoEx2)
print("Resultado da criação da matriz de incidências (versão em que frequência = nº de vezes que aparece na coleção):")
print(resultado2Ex2)
print('\n')

resultado22Ex2 = criarMatrizIncidenciasV2(resultadoEx2)
print("Resultado da criação da matriz de incidências 2 (versão em que frequência = nº de vezes que aparece em cada documento):")
print(resultado22Ex2)
print("\n")

print("Resultado da recuperação dos documentos dado uma consulta, operação, matriz de incidências e coleção:")
docsRecuperados = recuperaDocumento(q2, 1, resultado22Ex2, M2, separadores2, stopwords2)
print(docsRecuperados)

Resultado da normalização e tokenização:
[['parasita', 'grande', 'vencedor', 'oscar', '2020', 'quatro', 'prêmios'], ['green', 'book', 'roma', 'bohemian', 'rhapsody', 'principais', 'vencedores', 'oscar', '2019'], ['oscar', '2020', 'confira', 'lista', 'completa', 'vencedores', 'parasita', '1917', 'foram', 'grandes', 'vencedores', 'noite'], ['boa', 'fase', 'oscar', 'sonha', 'jogar', 'copa', 'mundo', 'rússia'], ['conheça', 'indicados', 'oscar', '2020', 'cerimônia', 'premiação', 'acontece', 'fevereiro'], ['oscar', 'schmidt', 'receberá', 'troféu', 'prêmio', 'brasil', 'olímpico', '2019', 'jogador', 'basquete', 'mais', 'pontos', 'jogos', 'olímpicos'], ['seleção', 'brasileira', 'vai', 'observar', '35', '40', 'jogadores', 'definir', 'lista', 'copa', 'américa'], ['oscar', '2020', 'saiba', 'escolha', 'jurados', 'votam'], ['bem', 'amigos', 'discute', 'lista', 'seleção', 'galvão', 'dá', 'recado', 'tite', 'cadê', 'luan'], ['ifal-maceió', 'convoca', 'aprovados', 'lista', 'espera', 'sisu', 'chamada', '